# 1-Generate Predictions using LangChain

- **Goal:** Prediction Recognition

- **Purpose:** To implement step 1 with sub steps of prediction recognition pipeline. See steps
    1. Generate predictions
        1. Create several prediction prompts templates
        2. Utilize open-source LLMs to generate predictions

- **Misc:**
    - `%store`: Cell magic will store the variable of interest so we can load in another notebook

Entities: You already have Representative Angela, Economist Dr. Henry, Meteorologist Nina, etc. Can we have at least one example per template? Other ex are analyst, expert, top executive,  senior level person, college student, research advisor, professional, military member, etc
- Alex, a health expert
- Angela Cortez, a member of Congress
- Dr. Anna Lee --> Anna Lee, Ph.D.

Future verb tense: A more balanced distribution/usage of the below
- Update "will likely" to will (without likely) may, would, be going to, should, etc (can google more)
- Can use rise/increase/as much as, fall/decrease/as little as, change, stay stable, high/low chance/probability/degree of, etc. (can google more)

Date: We write the date differently in many cases below. Want this to be represented in the dataset.  Try to  disperse the same usage to other templates and other domains. For example: T2 for Health all have [Month, date, year] and similarly for T5 in Health [Month Year] are all grouped within one template.  See example dates below
- (1) Wednesday, August 21, 2024; (2) Wed, August 21, 2024; (3) 08/21/2024; (4) 08/21/2024; (5) 21/08/2024; (6) 21 August 2024; (7) 2024/08/21; (8) 2024-08-21; (9) August 21, 2024; (10) Aug 21, 2024; (11) 21 August 2024, (12) 21 Aug 2024, Q3 of 2027, 2029 of Q3, etc (with removing day of week), first week of Fall, Q2 of 2033, 2045 Q3, etc.
- Differentiate the month, date, and year. 

In [1]:
import os, sys

import pandas as pd

from tqdm import tqdm
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from log_files import LogData
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

## Text Generation Models

In [2]:
tgmf = TextGenerationModelFactory()

llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

# gpt_35_turbo_generation_model = tgmf.create_instance('gpt-3.5-turbo')
# gpt_4_o_generation_model = tgmf.create_instance('gpt-4o')
mixtral_87b_instruct_generation_model = tgmf.create_instance('mixtral-8x7b-instruct') 
kokoro_generation_model = tgmf.create_instance('kokoro') 
llama_318b_generation_model = tgmf.create_instance('llama-3.1-8b-instruct')  
mistral_7b_generation_model = tgmf.create_instance('mistral-7b-instruct')  

# dsp_v2_7b_generation_model = tgmf.create_instance('DeepSeek-Prover-V2-7B') 


## Base Templates for Domain Predictions

In [3]:
full_prediction_template = """{prediction_properties}

{prediction_requirements}

{prediction_templates}

{prediction_examples}
"""

full_prediction_prompt = PromptTemplate.from_template(full_prediction_template)

In [4]:
prediction_properties_template = """A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_a>), where it consists of the following four properties:

    1. <p_s>, any source entity in the {prediction_domain} domain.
        - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc), civilian.
        - Can only be an organization that is associated with the {prediction_domain} prediction.
    2. <p_t>, any target entity in the {prediction_domain} domain.
	    - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc).
        - Can only be an organization that is associated with the {prediction_domain} prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. <p_a>, {prediction_domain} prediction attribute.
        - Characteristics of a domain-specific attributes such as various quantifiable metrics relevant to the {prediction_domain} domain.
        - Some examples are {prediction_domain_attribute}.  
"""
prediction_properties_prompt = PromptTemplate.from_template(prediction_properties_template)

In [5]:
prediction_requirements_template = """{prediction_domain} requirements to use for each prediction:

    - Should be based on real-world {prediction_domain} data and not hallucinate.
    - Only a simple sentence (prediction) (and NOT compounding using "and" or "or").
    - Should diversify all four properties of the prediction (<p>) as in change and not use same for <p_s>, <p_t>, <p_d>, <p_a>.
    - Should use synonyms to predict such as forecasts, speculates, foresee, envision, etc., and not use any of them more than ten times.
    - The prediction should be unique and not repeated.
    - Do not number the predictions.
    - Do not say, "As the {prediction_domain}, I will generate company-based {prediction_domain} predictions using the provided templates." or anything similar.
    - Use the five different templates and examples provided.
    - Change how the current date (<p_d>) written in the prediction with examples of (1) Wednesday, August 21, 2024; (2) Wed, August 21, 2024; (3) 08/21/2024; (4) 08/21/2024; (5) 21/08/2024; (6) 21 August 2024; (7) 2024/08/21; (8) 2024-08-21; (9) August 21, 2024; (10) Aug 21, 2024; (11) 21 August 2024, (12) 21 Aug 2024, Q3 of 2027, 2029 of Q3, etc (with removing day of week).
    {domain_requirements}
    - Do not say, "Here are {predictions_N} unique {prediction_domain} predictions based on the provided templates and examples:" in the prompt.
    - Do not use any of the examples in the prompt.
    - In front of every prodiction, put the template number in the format of "T1:", "T2:", etc. and do not number them like "1.", "2.", etc. Should have template number and generated prediction matching.
    - Do not put template number on line by itself. Always pair with a prediction.
    - Disregard brackets: "<>"
    - Should never say "Here are {predictions_N} unique {prediction_domain} predictions based on the provided templates and examples:" or "Note: I've made sure to follow the guidelines and templates provided, and generated unique predictions that meet the requirements."
    - Do not use person name of entity name more than once as in don't use name Joe as both the <p_s> and <p_t>, unless like Mr. Sach and Goldman Sach or Mr. Sam Walton and Sam's Club, etc.
    - The source entity (<p_s>) is rarely the same as the target entity (<p_t>) and if same, the <p_s> is making a prediction on itself in the <p_t>.
    - Should variate the slope of rise/increase/as much as, fall/decrease/as little as, change, stay stable, high/low chance/probability/degree of, etc.
    - Should variate the prediction verbs such as will, would, be going to, should, etc.
"""
prediction_requirements_prompt = PromptTemplate.from_template(prediction_requirements_template)

In [6]:
prediction_templates_template = """Here are some {prediction_domain} templates:

    - {prediction_domain} template 1: <p_s> forecasts that the <p_a> at <p_t> potentially decrease in <p_d>.
    - {prediction_domain} template 2: On <p_d>, <p_s> speculates the <p_a> at <p_t> will likely increase.
    - {prediction_domain} template 3: <p_s> predicts on <p_d>, the <p_t> <p_a> may rise.
    - {prediction_domain} template 4: According to <p_s>, the <p_a> at <p_t> would fall in <p_d>.
    - {prediction_domain} template 5: In <p_d>, <p_s> envisions that <p_t> <p_a> has some probability to remain stable.
    - {prediction_domain} template 6: <p_t> <p_a> should stay same <p_d>, according to <p_s>. 

"""
prediction_templates_prompt = PromptTemplate.from_template(prediction_templates_template)

In [7]:
prediction_examples_template = """Here are some examples of {prediction_domain} predictions:
{domain_examples}

With the above, generate a unique set of {predictions_N} predictions per template following the examples. Think from the perspective of an {prediction_domain} analyst, expert, top executive, or senior level person and even a college student, professional, research advisor, etc."""
prediction_examples_prompt = PromptTemplate.from_template(prediction_examples_template)

In [8]:
prediction_input_prompts = [
    ("prediction_properties", prediction_properties_prompt),
    ("prediction_requirements", prediction_requirements_prompt),
    ("prediction_templates", prediction_templates_prompt),
    ("prediction_examples", prediction_examples_prompt),
]

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prediction_prompt, pipeline_prompts=prediction_input_prompts
)

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_57393/3566428802.py:8: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


## Specific Templates for Domain Predictions

- For now, generating 1 prediction per template. From here, I'll try 3 and increase by increments/multiples of 3.

- With 1 prediction per template,
    - 1 prediction per template x 6 examples per domain so 6 predictions per domain
    - 6 predictions per domain x 4 domains = 24 predictions per model
    - 24 predictions per model x 2 models = 48 predictions across all models
    - 48 predictions across all models x 2 batches = 96 across all batches

In [9]:
examples_per_template = 1
generate_N_predictions_per_template = 1 * examples_per_template

- All
1. `will` to `may`
2. Use different entity names
    1. `Detravious`, `Morgan Stanley`, etc too many times
3. 

### Template for Financial Predictions

In [10]:
financial_attributes = """stock price, net profit, revenue, operating cash flow, research and development expenses, operating income, gross profit."""
financial_requirements = """- Should be based on real-world financial earnings reports.
    - Suppose the time when $p$ was made is during any earning season.
    - Include stocks from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc.
    - Include the US Dollar sign ($) before or USD after the amount of the financial attribute."""

financial_examples = """
   - financial examples for template 1:
    1. Detravious, an investor forecasts that the stock price at Apple will decrease in 2025 Q1.
    2. Ava Lee predicts that the operating cash flow at ExxonMobil should decrease in 03/21/2025 to 08/21/2025.
    3. Detravious, an investor predicts that the stocks he has will increase in 4/08/21.
 - financial examples for template 2:
    1. On March 15, 2025 to March 16, 2026, Goldman Sachs speculates that the interest rates at the Federal Reserve could rise.
    2. On April 2, 2000, Morgan Stanley forecasts that the stock value at Tesla should fall.
    3. On 1/23/2012, Chase analysts foresee that their stock prices may increase.
 - financial examples for template 3:
    1. Morgan Stanley anticipates that on 3/2/2025, the NASDAQ composite index could climb moderately.
    2. BlackRock foresees that on April 22, 2025, the value of Bitcoin has a high probability of rising sharply.
    3. Morgan Stanley predicts that on the 3 of May, 2025, there stock price might rise.
 - financial examples for template 4:
    1. According to Chase Bank, the expected returns at emerging market equities could fall in May 2035.
    2. According to Ryan, the projected revenue at Meta Platforms should fall in Q2 2025.
    3. According to Apple, the trading volume it has will increase in 1/2/2020.
 - financial examples for template 5:
    1. In 4/16/2025, Wells Fargo expects that U.S. Treasury yields might stay stable.
    2. In 5/5/2025, Bob envisions that the inflation rate in Wells Fargo could stay stable.
    3. In 7/5/2026, Bob predicts that the stocks he has will stay stable.
 - financial examples for template 6:
    1. Apple stock price will decrease in February 2025, according to Roger.
    2. The NASDAQ index is expected to rise in June 2025, according to Bank of America.
    3. Roger foresees the stock price increasing in July 2025, according to his projections.
 """

financial_input_dict = {
    "prediction_domain": "financial",
    "prediction_domain_attribute": financial_attributes,
    "domain_requirements": financial_requirements,
    "domain_examples": financial_examples,
    "predictions_N": generate_N_predictions_per_template
}
financial_prompt_output = pipeline_prompt.format(**financial_input_dict)
print(financial_prompt_output)


A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_a>), where it consists of the following four properties:

    1. <p_s>, any source entity in the financial domain.
        - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc), civilian.
        - Can only be an organization that is associated with the financial prediction.
    2. <p_t>, any target entity in the financial domain.
	    - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc).
        - Can only be an organization that is associated with the financial prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How fa

###  Template for Health Predictions


Want these tied to specific campaigns, regions, or health studies? Yeah, that's a good idea.  In regions, could include national, state, and local.  Be sure with the health study ex that it's still a prediction instead of it being data already collected and analyzed. 

- Template: According to <p_s>, the <p_a> at <p_t> would fall in <p_d>.
- Mappings: According to a <study by UF>, the <obesity rate> in <Gainesville, FL> should rapidly decrease between <9.8.2026 to 12.19.2026> (don't include in code; only showing relation to template. 
- Example: According to  study by UF, the obesity rate in Gainesville would  fall between 9.8.2026 to 12.19.2026

I'm cool with some examples not  exactly matching the template. The only exact matchings should be the variables. Every other word can be a synonym.

In [11]:
health_attributes = """obesity rates, prevalence of chronic illnesses, average physical activity levels, nutritional intake, etc."""
health_requirements = """- Should be based on real-world health reports.
    - Suppose the time when $p$ was made is during any season such as flu season, allergy season, pandemic, epidemic, etc.
    - Include reports from all Health organization, researcher, doctor, physical therapist, physician assistant, nurse practictioners, fitness expert, etc."""

health_examples = """
- health examples for template 1:
    1. I predicts that the obesity rates at the national level should fall in late 2025.
    2. CDC forecasts that the prevalence of chronic illnesses at urban health centers will decrease in Q3 2030.
    3. Chase Thinks that the average physical activity levels he does may increase in 2035.
- health examples for template 2:
    1. On 1/2/2015, the CDC speculates that the average physical activity levels at U.S. high schools will change.
    2. On June 1, 2035, Sam speculates that the nutritional intake at rural clinics in America may increase.
    3. On the 1st of July, 2025, Sam predicts that the obesity rates at the national level might decrease.
- health examples for template 3:
    1. The NIH predicts that on 6/22/2015, public engagement in preventative health screenings will decline.
    2. Alex suspects that on June 15, 2075, the obesity rates at the national level could decrease.
    3. Talon envisions that on 3, January 2025, the average physical activity levels for him potentially change.
- health examples for template 4:
    1. According to the Study conducted at UF, the obesity rates at U.S. elementary schools might fall in Fall 2035.
    2. According to the Me, the average sugar consumption at public school cafeterias will fall in 9/1/2025.
    3. According to James Study of physical activity, the average physical activity levels for him could fall in  Q2, 2029.
- health examples for template 5:
    1. In 5/2/2025, Dr. Maria Thompson envisions that national obesity rates could decrease.
    2. In August 2054, Professor James Liu envisions that average physical activity levels among teenagers will increase.
    3. In Q3 2026, Dr. Aisha Reynolds envisions that her nutritional intake can potentially stay stable.
- health examples for template 6:
    1. Physical activity levels among seniors could fall in 8/7/2015, according to Dr. Elena Morales study.
    2. Nutritional awareness in Asia will become more apparant in the 3rd of September 2014, according to Me.
    3. Sarah's health screening participation should rise in late 2024 Q3, according to Sarah.

Want these tied to specific campaigns, regions, or health studies?
"""

health_input_dict = {
    "prediction_domain": "health",
    "prediction_domain_attribute": health_attributes,
    "domain_requirements": health_requirements,
    "domain_examples": health_examples,
    "predictions_N": generate_N_predictions_per_template
}

health_prompt_output = pipeline_prompt.format(**health_input_dict)


###  Template for Policy Predictions

Change <e_s> to be first person in some cases

- T1. 3. Angela Cortez forcasts that data privacy laws in the technology sector will likely stay stable in the public, in late 2025, according to expert Angela Cortez. 
    - I forcast that data privacy laws in the technology sector will likely stay stable in the public, in late 2025, according to expert Angela Cortez.

- T6. 3. Rachel Alvarez's voter participation in local elections will likely stay the same in November 2025, according to Rachel Alvarez.
    - My voter participation in local elections will likely stay the same in November 2025, according to Rachel Alvarez.

In [12]:
policy_attributes = """election outcomes, economic reforms, legislative impacts."""
policy_requirements = """- Should be based on real-world policy reports.
    - Suppose the time when $p$ was made is during an election cycle or non-election cycles.
    - Include policies & laws, from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc."""

policy_examples = """
    - policy examples for template 1:
    1. Election outcomes in key swing states should rise in national importance in the 1st of November 2011, according to analyst Rachel Lin.
    2. Economic reforms in the energy sector will increase in visibility in Q3 2021, according to Dr. Marcus Grant from the Brookings Institution.
    3. I forcast that data privacy laws in the technology sector could stay stable in the public, in late 2025, according to expert Angela Cortez.
 - policy examples for template 2:
    1. On 4/5/2029, the Brookings Institution speculates that voter turnout at battleground states might stay stable.
    2. On May 12, 2020, the International Monetary Fund speculates that investment activity at emerging markets will rise.
    3. On the 1st of June, 2025, policy analyst Rachel Kim speculates that regulatory scrutiny in Rachels large tech firms may decrease.
 - policy examples for template 3:
    1. Representative Angela Brooks predicts that on October 15, 2027, the defense budget allocation potentially can be stay stable.
    2. Economist Dr. Henry Zhao predicts that on 6/4/2026, the corporate tax rate in the finance sector should rise.
    3. Senator Michael Greene predicts that on 3, November 2025, his voter engagement in suburban districts will fall.
 - policy examples for template 4:
    1. According to Senator Alicia Ramirez, the public trust at federal institutions may increase in late 2015.
    2. According to Thomas Nguyen, the investment confidence at the real estate sector will fall in Q3 2035.
    3. According to policy advisor Natalie Chen, the employment rate at at her company firms shall stay stable in September 2025.
 - policy examples for template 5:
    1. In 3/2/2023, Senator Jordan Ellis envisions that healthcare spending could stay stable.
    2. In June 2025, economist Dr. Priya Nandakumar envisions that inflation rates in the consumer staples sector will increase.
    3. In Q2 of 2021, policy strategist Kevin Adler envisions that his defense contract approvals may decline.
 - policy examples for template 6:
    1. Renewable energy investments are expected to inflate in Q3 2025, according to Dr. Elena Foster.
    2. Healthcare subsidies will decrease in September 2055, according to Senator Marcus Lee.
    3. My voter participation in local elections may stay the same in 11/3/2022, according to Rachel Alvarez.
"""
policy_input_dict = {
    "prediction_domain": "policy",
    "prediction_domain_attribute": policy_attributes,
    "domain_requirements": policy_requirements,
    "domain_examples": policy_examples,
    "predictions_N": generate_N_predictions_per_template
}
policy_prompt_output = pipeline_prompt.format(**policy_input_dict)

###  Template for Weather Predictions

I updated <p_d> for: 
- T2 1.
- T2 3.

In [13]:
weather_attributes = """temperature, precipitation, wind speed, humidity, etc."""
weather_requirements = """- Should be based on real-world weather reports.
    - Suppose the time when $p$ was made is during any season and any location (ie: Florida known for hurricanes, California known for wildfires, etc).
    - Include reports from all meteorologists, weather organizations, or any type of weather entity.."""

weather_examples = """
- weather examples for template 1:
    1. The National Weather Service forecasts that the precipitation levels at Miami may increase in September 5, 2025.
    2. AccuWeather forecasts that the humidity at Phoenix will stay stable in early fall 2025.
    3. Sam a weather expert forecasts that the wind speed at his house could decrease in 3/4/2025.
- weather examples for template 2:
    1. On 1/1/2025, Meteorologist Lisa Park speculates that the temperature at Los Angeles will rise.
    2. On 2025 of June, Dr. Mark Williams speculates that the humidity at Houston shall become lower.
    3. On July 5, 2025, San Francisco's meteorological team speculates that the wind speed in San Francisco potentially could stay stable.
- weather examples for template 3:
    1. Anna Lee, PH.D predicts that on May 2025, the temperature at Denver can decrease.
    2. Meteorologist John Roberts predicts that on July 1, 2025, the wind speed at New York will rise.
    3. The Miami Weather Bureau predicts that on 7/2/2025, the humidity at Miami might stay the same.
- weather examples for template 4:
    1. According to Dr. Linda Garcia, the temperature at Boston shall increase in November 2025.
    2. According to Meteorologist Jake Wilson, the precipitation levels at Seattle will stay stable in January 1, 2025.
    3. According to Dylan, the wind speed at Dylans home may fall in 10/5/2025.
- weather examples for template 5:
    1. In 3, December 2025, Meteorologist Claire Thompson envisions that the temperature at Chicago could stay increase.
    2. In 5/2/2025, Dr. Robert Harris envisions that the humidity at Phoenix will stay stable.
    3. In the month of July, the San Francisco Weather Bureau envisions that the precipitation levels in San Francisco should fall.
- weather examples for template 6:
    1. Temperature in Las Vegas has a low chance to rise in July 2025, according to Meteorologist Nina Patel.
    2. Humidity in Houston will decline in 2025, Aug 3, according to Dr. Kevin Morales.
    3. Wind speed in Miami could stay stable in 10/1/2025, according to the Miami Weather Bureau.
"""

weather_input_dict = {
    "prediction_domain": "weather",
    "prediction_domain_attribute": weather_attributes,
    "domain_requirements": weather_requirements,
    "domain_examples": weather_examples,
    "predictions_N": generate_N_predictions_per_template
}
weather_prompt_output = pipeline_prompt.format(**weather_input_dict)

### Generate Sports Predictions

In [14]:
sport_attributes = """score, touchdown, goal, points, win, lose, etc."""
sport_requirements = """- Should be based on real-world sports.
    - Suppose the time when $p$ was made is during any season of sports.
    - Include reports from all sports professionals, coaches, or any type of sport entity.."""

sport_examples = """
- sport examples for template 1:
    1. Coach Lisa Martinez predicts that the touchdown rate at the Miami Dolphins will fall in 2020 of October.
    2. Analyst Mark Johnson forecasts that the goal average at Manchester United will stay the same in November 2025.
    3. Ryan forecasts win percentage he has for soccer will go up in 12/25/2016.
- sport examples for template 2:
    1. On Sep 20, 2100, Coach Maria Lopez suggests that the score average at the Chicago Bulls is climbing.
    2. On 9/12/2025, Analyst David Kim anticipates the touchdown rate at the Kansas City Chiefs will likely surge.
    3. On October 8, 2123, Detravious foresees that the win probability he has for rugby is expected to trend downward.
- sport examples for template 3:
    1. Coach Elena Ruiz predicts on 9/22/2025, the goal count at Real Madrid will climb.
    2. Analyst Marcus Lee forecasts that on Sep 30, 2055, the point average at the Golden State Warriors will be higher.
    3. George Jr. estimates that on October 15, 2035, the win ratio for games he has will disimprove.
- sport examples for template 4:
    1. According to Coach Sarah Nguyen, the scoring average at the Dallas Mavericks is expected to dip in Sep 2021.
    2. According to Analyst Trevor Simmons, the touchdown rate at the Green Bay Packers will increase in 10/2025.
    3. According to Manchester United, the win percentage at Manchester United is projected to drop in October 2034.
- sport examples for template 5:
    1. In 9/2025, Coach Miguel Torres envisions that the goal average at Paris Saint-Germain will hold steady.
    2. In October 2056, Analyst Fiona Bennett anticipates that the win rate at the Toronto Raptors will decrease slightly.
    3. In Sep 2086, Calvin foresees that the points per game he has in football will gradually increase.
- sport examples for template 6:
    1. The goal count at Liverpool FC will surge in Sep 2012, according to Coach Daniel Alvarez.
    2. The win percentage at the Chicago Bears will taper off in October 2025, according to Analyst Priya Sharma.
    3. The scoring average on Arnolds footbal team will remain steady in 10/2034, according to Arnold.
"""
sport_input_dict = {
    "prediction_domain": "sport",
    "prediction_domain_attribute": sport_attributes,
    "domain_requirements": sport_requirements,
    "domain_examples": sport_examples,
    "predictions_N": generate_N_predictions_per_template
}
sport_prompt_output = pipeline_prompt.format(**sport_input_dict)

### Generate Miscellaneous Predictions

In [15]:
miscellaneous_attributes = """These attributes will take in any random attribute relating ot any real world situation."""
miscellaneous_requirements = """These attributes will take in any random attribute relating ot any real world situation.
    - Suppose the time when $p$ was made is during any season or part of the year.
    - Include any type of entity.."""

miscellaneous_examples = """
- miscellaneous examples for template 1:
    1. Professor Laura White forecasts that the final exam scores at Crestwood Academy will drop in May 2023.
    2. Chef Alberto Reyes predicts that the sweetness level in the bakery’s pastries will decline in fall 2027.
    3. Gaming expert Jordan Lee foresees that the chances of rolling a seven at game night will decrease in October 2025.
- miscellaneous examples for template 2:
    1. On 9/18/2000, Professor Maria Jackson speculates that the GPA at Eastwood College will go up.
    2. On November 2, 2020, Chef Gabrielle Moreau suggests that the saltiness in the soup at Le Petit Bistro will rise.
    3. On December 10, 2025, Dungeons & Dragons master Oliver Cheng speculates that the chance of rolling a natural 20 will increase.
- miscellaneous examples for template 3:
    1. Dr. Sarah McDonald predicts that on Dec 15, 2029, the average student grade at Jefferson High will rise.
    2. Coach Tony Roberts predicts that on 10/25/2015, the batting average at the Boston Red Sox will improve.
    3. Sarah predicts that on January 5, 2026, the odds of drawing a joker from the deck will rise.
- miscellaneous examples for template 4:
    1. According to Coach Andre Collins, the three-point shooting percentage at Phoenix Suns will fall in January 2010.
    2. According to Chef Emily Gonzalez, the bitterness of the coffee at The Brew Café will likely decrease in Winter 2012.
    3. According to game analyst Tom Spencer, the chance of rolling doubles in the Monopoly game will trend upward in September 2028.
- miscellaneous examples for template 5:
    1. In fall 2029, Professor Kim Tansley envisions that the test scores at Riverbrook University will stay stable
    2. In 12/2019, Chef Michael Harris envisions that the spice level at Curry King will remain steady.
    3. In March of 2026, ChatGPT envisions that the dice rolling outcomes for a 6 will stay consistent.
- miscellaneous examples for template 6:
    1. Student performance will improve in Spring 2028, according to Professor B.T.
    2. The flavor of the tomato sauce will increase in October 2015, according to Chef Veronica Miller.
    3. FD's chances of rolling a 12 will stay the same on 1/11/2025, according to FD.
"""
miscellaneous_input_dict = {
    "prediction_domain": "miscellaneous",
    "prediction_domain_attribute": miscellaneous_attributes,
    "domain_requirements": miscellaneous_requirements,
    "domain_examples": miscellaneous_examples,
    "predictions_N": generate_N_predictions_per_template
}
miscellaneous_prompt_output = pipeline_prompt.format(**miscellaneous_input_dict)

## Generate Predictions

In [16]:
N_batches = 1
# text_generation_models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, 
#                           llama_8b_8192_generation_model, gpt_35_turbo_generation_model, gpt_4_o_generation_model, 
#                           mixtral_87b_instruct_generation_model]
# text_generation_models_navigator = [mixtral_87b_instruct_generation_model]
text_generation_models_navigator = [llama_318b_generation_model, mistral_7b_generation_model]

prediction_domains = ["finance", "health", "policy", "weather", "sport", "miscellaneous"]
prediction_prompt_outputs = {
    "finance": financial_prompt_output,
    "health": health_prompt_output,
    "policy": policy_prompt_output,
    "weather": weather_prompt_output,
    "sport": sport_prompt_output,
    "miscellaneous": miscellaneous_prompt_output,
}
prediction_label = 1

batched_predictions_df = tgmf.batch_generate_data(N_batches=N_batches, 
                                text_generation_models=text_generation_models_navigator, 
                                domains=prediction_domains,
                                prompt_outputs=prediction_prompt_outputs,
                                sentence_label=prediction_label)

  0%|          | 0/1 [00:00<?, ?it/s]

===================================== Batch 0 ===============================================
finance --- llama-3.1-8b-instruct --- NAVI_GATOR
    generates: T1: Morgan Stanley forecasts that the stock price at Amazon will decrease in 2027 Q3.

T2: On 08/21/2024, Goldman Sachs speculates that the operating cash flow at Microsoft will likely increase.

T3: Morgan Stanley predicts on 2024-08-21, the stock value at Alphabet may rise.

T4: According to Ryan, the projected revenue at Facebook would fall in Q4 2025.

T5: In 2024/08/21, Wells Fargo envisions that the inflation rate in the US could stay stable.

T6: Alphabet stock price will stay same in Q2 2027, according to Google's projections.
> /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/text_generation_models.py(181)generate_predictions()
    180         # Parse the raw text into structured data (assuming a consistent format)
--> 181         predictions = []
    182         for line in raw_text.s

  0%|          | 0/1 [00:11<?, ?it/s]


In [ ]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
batched_predictions_df